# Data Wrangling Summary



### Gather
* Depending on the source of your data, and what format it's in, the steps in gathering data vary.
* High-level gathering process: obtaining data (downloading a file from the internet, scraping a web page, querying an API, etc.) and importing that data into your programming environment (e.g., Jupyter Notebook).

### Assess

Assess data for:
* **Quality:** issues with content. Low quality data is also known as dirty data.
* **Tidiness:** issues with structure that prevent easy analysis. Untidy data is also known as messy data. Tidy data requirements:
  - Each variable forms a column.
  - Each observation forms a row.
  - Each type of observational unit forms a table.

### Types of assessment:
* **Visual assessment:** scrolling through the data in your preferred software application (Google Sheets, Excel, a text editor, etc.).
* **Programmatic assessment:** using code to view specific portions and summaries of the data (pandas' `head`, `tail`, and `info` methods, for example).

### Clean

### Types of cleaning:
* Manual (not recommended unless the issues are single occurrences)
* Programmatic
The programmatic data cleaning process:
    
    * **Define:** convert our assessments into defined cleaning tasks. These definitions also serve as an instruction list so others (or yourself in the future) can look at your work and reproduce it.
    * **Code:** convert those definitions to code and run that code.
    * **Test:** test your dataset, visually or with code, to make sure your cleaning operations worked.
      
  Always make copies of the original pieces of data before cleaning!

### Reassess and Iterate
After cleaning, always reassess and iterate on any of the data wrangling steps if necessary.

### Store (Optional)
Store data, in a file or database for example, if you need to use it in the future.

## Gathering



In [0]:
import zipfile
import pandas as pd

In [0]:
with zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/Data Wrangling/armenian-online-job-postings.zip','r') as myzip:
  myzip.extractall()

In [0]:
df=pd.read_csv('/content/online-job-postings.csv')

In [0]:
df

In [0]:
df.head()
df.tail()
# df.info()


In [0]:
import requests
url= 'https://pandas.pydata.org/pandas-docs/stable/reference/index.html'
response = request.get(url)
#save html to file
with open("file_name.html", mode ='wb') as file:
  file.write(response.content)

In [0]:
# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
folder = 'rt_html'
for movie_html in os.listdir(folder):
    with open(os.path.join(folder, movie_html)) as file:
        soup= BeautifulSoup(file,'lxml')
        title = soup.find('title').contents[0][:-len(' - Rotten Tomatoes')]
        audience_score = soup.find('div',class_='audience-score meter').find('span').contents[0][:-1]
        num_audience_ratings = soup.find('div', class_ ='audience-info hidden-xs superPageFontColor')
        num_audience_ratings=num_audience_ratings.find_all('div')[1].contents[2].strip().replace(',', '')

        # Note: a correct implementation may take ~15 seconds to run
        
        
        # Append to list of dictionaries
        df_list.append({'title': title,
                        'audience_score': int(audience_score),
                        'number_of_audience_ratings': int(num_audience_ratings)})
df = pd.DataFrame(df_list, columns = ['title', 'audience_score', 'number_of_audience_ratings'])

In [0]:
df_solution = pd.read_pickle('df_solution.pkl')
df.sort_values('title', inplace = True)
df.reset_index(inplace = True, drop = True)
df_solution.sort_values('title', inplace = True)
df_solution.reset_index(inplace = True, drop = True)
pd.testing.assert_frame_equal(df, df_solution)

In [0]:
# Make directory if it doesn't already exist
folder_name = 'ebert_reviews'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [0]:
ebert_review_urls = ['https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9900_1-the-wizard-of-oz-1939-film/1-the-wizard-of-oz-1939-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991e_99-the-godfather-part-ii/99-the-godfather-part-ii.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991e_100-battleship-potemkin/100-battleship-potemkin.txt']

In [0]:
# downloading text files
for url in ebert_review_urls:
    response=requests.get(url)
    with open(os.path.join(folder_name,url.split('/')[-1]), mode ='wb') as file:
        file.write(response.content)

In [0]:
import glob

# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
for ebert_review in glob.glob('ebert_reviews/*.txt'):
    with open(ebert_review, encoding='utf-8') as file:
        title = file.readline()[:-1]
        # Your code here
        review_url = file.readline()[:-1]
        review_text = file.read()

        # Append to list of dictionaries
        df_list.append({'title': title,
                        'review_url': review_url,
                        'review_text': review_text})
df = pd.DataFrame(df_list, columns = ['title', 'review_url', 'review_text'])

In [0]:
import pandas as pd
import wptools
import os
import requests
from PIL import Image
from io import BytesIO
# List of dictionaries to build and convert to a DataFrame later
df_list = []
image_errors = {}
for title in title_list:
    try:
        # This cell is slow so print ranking to gauge time remaining
        ranking = title_list.index(title) + 1
        print(ranking)
        page = wptools.page(title, silent=True)
        # Your code here (three lines)
        images = page.get().data['image']
        # First image is usually the poster
        first_image_url = images[0]['url']
        r = requests.get(first_image_url)
        # Download movie poster image
        i = Image.open(BytesIO(r.content))
        image_file_format = first_image_url.split('.')[-1]
        i.save(folder_name + "/" + str(ranking) + "_" + title + '.' + image_file_format)
        # Append to list of dictionaries
        df_list.append({'ranking': int(ranking),
                        'title': title,
                        'poster_url': first_image_url})
    
    # Not best practice to catch all exceptions but fine for this short script
    except Exception as e:
        print(str(ranking) + "_" + title + ": " + str(e))
        image_errors[str(ranking) + "_" + title] = images

In [0]:
from sqlalchemy import create_engine
# Create SQLAlchemy Engine and empty bestofrt database
# bestofrt.db will not show up in the Jupyter Notebook dashboard yet
engine = create_engine('sqlite:///bestofrt.db')
# Store cleaned master DataFrame ('df') in a table called master in bestofrt.db
# bestofrt.db will be visible now in the Jupyter Notebook dashboard
df.to_sql('master', engine, index=False)
Read the brand new data in that database back into a pandas DataFrame.
df_gather = pd.read_sql('SELECT * FROM master', engine)

## Assessing
**Assessing** your data is the second step in data wrangling. When assessing, you're like a detective at work, inspecting your dataset for two things: data quality issues (i.e. content issues) and lack of tidiness (i.e. structural issues).

Assessing is the precursor to cleaning. You can't clean something that you don't know exists! In this lesson, you'll learn to identify and categorize common data quality and tidiness issues. This lesson is the shortest and most "hands-off" code-wise of all four in the course because of the passive nature of assessing relative to gathering and cleaning. We have tried to include quizzes wherever possible.

You can assess data for:

- **Quality:** issues with content. Low quality data is also known as dirty data.
- **Tidiness:** issues with structure that prevent easy analysis.

Untidy data is also known as messy data. Tidy data requirements:
- Each variable forms a column.
- Each observation forms a row.
- Each type of observational unit forms a table.

...using two types of assessment:

1. Visual assessment: scrolling through the data in your preferred software application (Google Sheets, Excel, a text editor, etc.).
2. Programmatic assessment: using code to view specific portions and summaries of the data (pandas' head, tail, and info methods, for example).


### Visual Assesment
#### Data Quality Dimensions

Data quality dimensions help guide your thought process while assessing and also cleaning. The four main data quality dimensions are:

- **Completeness:** do we have all of the records that we should? Do we have missing records or not? Are there specific rows, columns, or cells missing?
- **Validity:** we have the records, but they're not valid, i.e., they don't conform to a defined schema. A schema is a defined set of rules for data. These rules can be real-world constraints (e.g. negative height is impossible) and table-specific constraints (e.g. unique key constraints in tables).
- **Accuracy:** inaccurate data is wrong data that is valid. It adheres to the defined schema, but it is still incorrect. Example: a patient's weight that is 5 lbs too heavy because the scale was faulty.
- **Consistency:** inconsistent data is both valid and accurate, but there are multiple correct ways of referring to the same thing. Consistency, i.e., a standard format, in columns that represent the same data across tables and/or within tables is desired.

### Programmatic Assesment
These are the programmatic assessment methods in pandas that you will probably use most often:

* .head (DataFrame and Series)
* .tail (DataFrame and Series)
* .sample (DataFrame and Series)
* .info (DataFrame only)
* .describe (DataFrame and Series)
* .value_counts (Series only)
* Various methods of indexing and selecting data (.loc and bracket notation with/without boolean indexing, also .iloc)